# Machine Learning project CS-433: NMR spectroscopy supervised learning



## Schedules:

* Week 10 (18-24 November): 
 * Tests of various linear models/simple NN on a 10% subset of data
* Week 11 (25-1 December):
 * Feature selection: being able to come with a good set of features
* Week 12 (2-8 December):
 * Start of big scale analysis with Spark, implementation of the models which perform well at small scale
* Week 13 (9-15 December):
 * Wrapping up
* Week 14 (16-22 December): 
 * 19th December: Deadline

In [ ]:
import os
import re
import pickle
import scipy.stats
import sklearn.metrics

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from itertools import combinations

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

## Log Book

We write here the log of the different techniques/improvements we add to the program: **cf log/models_log.txt** for the different models already tried and their results.

## Pipeline

In [ ]:
#pipeline graph coming soon

In [ ]:
tot_data_X = np.load('data/CSD-10k_H_fps_1k_MD_n_12_l_9_rc_3.0_gw_0.3_rsr_1.0_rss_2.5_rse_5.npy',mmap_mode='r')
tot_data_y = np.load('data/CSD-10k_H_chemical_shieldings.npy',mmap_mode='r')

In [ ]:
def load_data(n_samples,tot_data_x = tot_data_X,tot_data_y = tot_data_y):
    
    N_SAMPLES = 3000
    DATA_LEN = tot_data_X.shape[0]
    DATA_COLS = tot_data_X.shape[1]

    #np.random.seed(14)
    mask_data = np.random.permutation(DATA_LEN)[:N_SAMPLES]

    data_X = tot_data_X[mask_data]
    data_y = tot_data_y[mask_data]
    X_train,X_test,y_train,y_test = train_test_split(data_X,data_y,test_size = 0.2)
    return X_train,X_test,y_train,y_test

## Data Vizualisation

In [ ]:
data_X_df = pd.DataFrame(data_X)
data_y_df = pd.DataFrame(data_y)
mask = np.random.permutation(DATA_COLS)[:9]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3)
fig.set_size_inches(11,11)
for ind,i in enumerate(mask):
    index = np.unravel_index(ind,(3,3))
    axes[index].ticklabel_format(style='sci',scilimits=(-3,4),axis='both')
    data_X_df.iloc[:,i].hist(ax = axes[index],bins = 50)
    #data_X_df.iloc[:,i].plot.box(ax = axes[index])

We see that the different features are scaled pretty differently, we might want to scale them beforehand. Since they don' look like following a gaussian, we'll apply min/max scaling: but in order to do so, we first need to get rid of the outliers thanks to one of the following methods
* Zscore: not adapted as our data might not be gaussian
* DBScan:
* Isolation Forest:

## Outliers detection

In [ ]:
clustering = DBSCAN(eps=0.3, min_samples=2).fit(X_train)

with np.printoptions(threshold=np.inf):
    print(clustering.labels_)

## Scaling

### Min/max Scaling

In [ ]:
def apply_scaler(scaler,X_train,X_test):
    X_train = minmx_scaler.fit_transform(X_train)
    X_test = minmx_scaler.transform(X_test)
    return X_train,X_test

## Dimensionality reduction

### PCA

In [ ]:
def plot_PCA(n_comp,X_train):
    pca = PCA(n_components = n_comp)
    pca.fit(X_train)
    plt.figure(1)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance')
    plt.show()
    
plot_PCA(70,X_train)

In [ ]:
def do_PCA(X_train,X_test,n):
    pca = PCA(n_components = n)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

## Models

### Linear Models

In [ ]:
def test_lin_model(reg, X_train,y_train,X_test,y_test):
    """
    Test a model reg which is expected to be already instantiated
    return score_train,score_test"""
    lin.fit(X_train,y_train)
    train_score = reg.score(X_train,y_train)
    test_score = reg.score(X_test,y_test)
    print("Obtained score on train set %2.2f " % train_score)
    print("Obtained score on test set %2.2f " % test_score)
    return train_score, test_score

## Neural Nets

## Main

Each cell here is meant to do a whole pipeline, from loading a certain number of samples, preprocessing etc. We keep using the R2 score as our metric

In [ ]:
X_train,X_test,y_train,y_test = load_data(10000)

In [ ]:
#Linear Regression
#do_PCA(X_train,X_test,40)
#minmx_scaler = MinMaxScaler()
#X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)
lin = LinearRegression(fit_intercept = True).fit(X_train,y_train)
train_score,test_score = test_lin_model(lin,X_train,y_train,X_test,y_test)

In [ ]:
#Ridge regression
#do_PCA(X_train,X_test,40)
#minmx_scaler = MinMaxScaler()
#X_train, X_test = apply_scaler(minmx_scaler,X_train,X_test)
rid = Ridge().fit(X_train,y_train)
train_score,test_score = test_lin_model(rid,X_train,y_train,X_test,y_test)